<a href="https://colab.research.google.com/github/GuerrSim96/Archimista_Ex/blob/main/Indice_dei_nomi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>INDICE DEI NOMI</h1><p align='right'><sub>(necessario quando viene prodotto l'inventario cartaceo)</sub></p>

In [ ]:
# @title <h2> Importazione dei pacchetti </h2>
!pip install PyMuPDF
import re
import os
import shutil as sh
import fitz

In [ ]:
# @title <h2> Impostazione parametri:
# @markdown <p> Inseriere il nome del file esportato da Archimista (con estensione .txt):</p>
input_file = "" # @param {type:"string"}
# @markdown <p> Inseriere il nome del file contenente l'inventario (con estensione .pdf):</p>
pdf = "" # @param {type:"string"}
# @markdown <p> Selezionare con il cursore il numero di sottoserie presenti nel file di input:</p>
tot_sserie = 6 # @param {type:"slider", min:1, max:20, step:1}
n_sserie, serie = 1, [] # Creazione del contatore e lista vuota
while n_sserie <= tot_sserie: # Ciclo while per creare liste
  output, lst_output, sserie= f'sottoserie_{n_sserie}.txt', [], [] # Creazione elemento lista e liste vuote
 # Funzione input() per inserire gli estremi delle sottoserie
  ttl_1 = input(f'inserisci il titolo del primo documento contenuto nella {str(n_sserie)}^ sottoserie: ')
  ttl_end = input(f'inserisci il titolo dell\'ultimo documento contenuto nella {str(n_sserie)}^ sottoserie: ')
 # Funzione lista.append(elemento) per riempire le liste
  sserie.append(ttl_1)
  sserie.append(ttl_end)
  serie.append(sserie)
  lst_output.append(output)
  n_sserie += 1 # Avanzamento del contatore

In [ ]:
# @title <h2> Estrazione: Nomi e Cognomi </h2>
part_cog = ['De', 'Di', 'La', 'Lo', 'Della', 'Del'] # Lista perticelle cognomi
for count, out in enumerate(lst_output): # Ciclo for: enumerate(lista) fornisce la posizione dell' elemento e l'elemento stesso
 # Aprertura file in modalità lettura e ottenimento del contenuto
  with open(input_file, 'r', encoding='utf-8') as file:
    lines = file.readlines()
 # Creazione variabili nulle e ottenimento dell'elemento i-esimo della lista
  start_index, end_index, n_lst = None, None, serie[count]
  for row, line in enumerate(lines): # Assegnazione del numero di riga alla variabile
    if f"<unittitle localtype='denominazione'>{n_lst[0]}</unittitle>" in line:
      start_index = row # Assegnazione del numero di riga alla variabile
    elif f"<unittitle localtype='denominazione'>{n_lst[1]}</unittitle>" in line:
      end_index = row # Assegnazione del numero di riga alla variabile
      break
  if start_index is not None and end_index is not None:
    selected_lines = lines[start_index:end_index + 1]
    with open(out, 'w', encoding='utf-8') as file_output: # Creazione di un file vuoto
      file_output.writelines(selected_lines) # Scrittura del file
 # Aprertura file in modalità lettura e ottenimento del contenuto
  with open(out, 'r') as read_file:
    lines = read_file.readlines()
    if count == 0:
      with open('sserie_merged.txt', 'w') as created_file:
        created_file.write(lines)
    else:
      with open('sserie_merged.txt', 'a') as add_to_file:
        add_to_file.write(lines)
# Creazione e ricerca del pattern
pattern, list_name = re.compile(r'<unittitle localtype=\'denominazione\'>(.*?)</unittitle>'), []
with open('sserie_merged.txt', 'r') as cleaning_file: # Creazione di un file vuoto
  lines = cleaning_file.readlines() # Scrittura del file
for line in lines: # Ciclo for per ricercare in ogni riga del file
  matches= re.findall(pattern, line)
for match_name in matches:
  clean_name_1 = ''.join(car for car in match_name if not (car.isdigit() or car == '.')) # Eliminazione di numeri e punteggiatura
  clean_name_2 = [word.lstrip() for word in clean_name_1] # Lambda per eliminazione degli spazi prima e dopo l'elemento
  list_name.append(clean_name_2) # Inserimento in lista il nome trovato
# @markdown <p>Generalmente sui documenti (e quindi nei titoli delle schede su Archimista) i soggetti vengono riportati come "NOME COGNOME", ma dato che la ricerca avviene tramite COGNOME bisogna riordinare i soggetti "COGNOME NOME".</p>
list_cog = [] # Creazione lista vuota
for person in list_name:
  lst_pers_1 = person.split() # Trasformazione di stringa in lista
  while 'bis' in lst_pers_1: # Ciclo while
    lst_pers_1.remove('bis') # Rimozione della particella "bis"
  if len(lst_pers_1) == 2: # Ciclo if: se la lista contiene 2 elementi
      lst_pers_1[0], lst_pers_1[1] = lst_pers_1[1], lst_pers_1[0] # Inverti l'Ordine degli elementi
  elif len(lst_pers_1) == 3: # se la lista contiene 3 elementi
    if set(part_cog) & set(lst_pers_1): # Ciclo if: se il cognome presenta una particella
      lst_pers_1 = lst_pers_1[1:] + [lst_pers_1[0]] # Sposta il primo elemento alla fine della lista
    else: # altrimenti marchia il soggetto
      lst_pers_1 = ['0'] + lst_pers_1
  else: # altrimenti marchia il soggetto
      lst_pers_1 = ['0'] + lst_pers_1
  clean_name_3 = ' '.join(lst_pers_1)  # Trasforma elemnto della lista in stringa separando ogni elemento da uno spazio
  list_cog.append(clean_name_3) # Inserimento dei nomi "permutati" all'interno di una lista
ordered_list_cog, last_output = [], 'INDICE_NOMI_DEFINITIVO.txt'
for cname in list_cog:
  set_cog = set(list_cog) # Eliminare dei duplicati dalla lista
  ordered_list_cog = sorted(set_cog) # Ordinare alfabetico degli elementi in lista

for c_n in ordered_list_cog:
  lst_pers_2 = line.split()
  if len(lst_pers_2) == 2:
      lst_pers_2[0], lst_pers_2[1] = lst_pers_2[1], lst_pers_2[0]
  elif len(lst_pers_2) == 3:
    if set(part_cog) & set(lst_pers_2):
      x=[]
      x.append(lst_pers_2[2])
      lst_pers_2 = x + lst_pers_2[:2]
      rev_name = ' '.join(lst_pers_2)
  lst_inverted.append(rev_name)
with open(last_output, 'w') as last:
  for n_c in lst_inverted:
    x = list(n_c)
    if '0' in x:
      continue
    else:
      inventory = fitz.open(pdf)
      for num in range(inventory.page_count):
        page = inventory[num]
        text = page.get_text()
        if line.strip() in text:
          page_num = []
          i = str(num + 1)  # L'indice è 0-based
          page_num.append(i)
          all_page = ", ".join(page_num)
          last.writelines(f'{line.strip()} {all_page}\n')

In [ ]:
#@title <h2> Esportazione del file contenente l'indice dei nomi e i file input associati.
folder = 'INDICE_NOMI' # Creazione variabile
if not os.path.exists(folder): # Ciclo if: se non esiste
  os.mkdir(folder) # Crea una cartella con il nome della variabile
  print(f'{folder} folder has been created') # Messaggio a schermo
else: # Altrimenti non creare la cartella
  print(f'{folder} folder already exists') # Messaggio a schermo
lst_export = [input_file, pdf, last_output] # Creazione lista
for file_text in lst_export: # Ciclo for: elemnti in lista
  sh.copy2(file_text, folder) # Copia dei file nella cartella
sh.make_archive(f'{folder}_COMPLETO', 'zip', folder) # Compressione della cartella
# Download della cartella compressa
from google.colab import files
files.download(f'{folder}_COMPLETO.zip')